First, import all the necessary libraries:
* liblibra_core - for general data types from Libra
* libra_py - for the normal modes module
* py3Dmol - for visualization

The output of the cell below will throw a bunch of warnings, but this is not a problem nothing really serios. So just disregard them.

In [13]:
from liblibra_core import *
from libra_py import *
import math
import py3Dmol

In the next stage - we will read in the MD trajectory stored in an XML file. 
This is done with the *read_md_data* function of the **QE_methods** module. This module focuses on processing data generated with Quantum Espresso (QE)  program.
The function returns the following data:
* R - coordinates of all atoms for all timesteps
* V - velocities of all atoms for all timesteps 
* A - accelerations of all atoms for all timesteps
* M - masses of all degrees of freedom (DOFs). 
* E - element labels of all atoms in the system

The present directory contains 3 files with MD trajectories for 3 systems: 
* x0.xml - Si8 cluster
* x0_co2.xml - CO2 molecule
* x0_h2.xml - H2 molecule

To do the calculations for the system of interest - uncomment one of the lines below and comment other 2. The selection of the system will of course affect how many modes will be available in the output after that.

In [14]:
#R, V, A, M, E = QE_methods.read_md_data("x0.xml")
R, V, A, M, E = QE_methods.read_md_data("x0_co2.xml")
#R, V, A, M, E = QE_methods.read_md_data("x0_h2.xml")

The following line sets up a parameters dictionary. The key-value pairs are as follows:

* visualize - determines whether we want to produce an xyz file that "visualizes" th selected normal modes. In the present example, we set this variable to 0, because we don't want those files. Instead, all the visualization will be done in this notebook with the help of py3Dmol library.
* verbosity - the level of messaging printed out in the noraml modes calculations. With the selection (0), we don't print out any messages to not spoil with output. This could be useful for debugging and such. 

With the present selection of the *visualize* variable, the rest of the parameters won't really matter, but I explain them here:
* prefix - the filename where the xyz "normal mode trajectory" will be printed out
* nsteps - how many steps of such visualization dynamics to do
* nperiods - for how many times to period the visualization dynamics
* scale - the factor that scales the normal modes' amplitudes

In [15]:
params = {"visualize":0, "verbosity":0, "prefix":"", "nsteps":100, "nperiods":1, "scale":250.0}

Now, we are ready to use the MD data which we have read from the XML file and compute the normal modes according to several methods. In these calculations all the parameters will be the same, except for one:

* cov_flag - this flag controls whether we want to use the property's fluctuation or the property itself. For instance, whether we'll be using the atomis velocities as they are of if the average velocities should be first subtracted from the instantaneous values. Note, the original papers describing the methods below talk about the atomic properties (e.g. velocities) themselves rather than about their fluctuations, so the **cov_flag = 0** would be the preferable choice. 

### Method 1:

Strachan, A. Normal Modes and Frequencies from Covariances in Molecular Dynamics 
    or Monte Carlo Simulation. *J. Chem. Phys.* **2003**, 120, 1-4.
    
In the following 2 calls *res* is a tuple (w, w_inv_cm, U_v,  w2, w2_inv_cm, U_a)

In [16]:
params.update({"cov_flag":0})
res00 = normal_modes.compute_cov( R, V, A, M, E, params)

Another option is to use data centered around the averages (fluctuations)

In [17]:
params.update({"cov_flag":1})
res01 = normal_modes.compute_cov( R, V, A, M, E, params)

### Method 2:
Pereverzev, A.; Sewell, T. D. Obtaining the Hessian from the Force Covariance Matrix:
    Application to Crystalline Explosives PETN and RDX. *J. Chem. Phys*. **2015**, 142, 134110.
    
In the following 2 calls *res* is a tuple (w_a, w_inv_cm, U_a)
Again, without centering:

In [18]:
params.update({"cov_flag":0})
res20 = normal_modes.compute_cov2( R, A, M, E, 300.0, params)

With the centering:

In [19]:
params.update({"cov_flag":1})
res21 = normal_modes.compute_cov2( R, A, M, E, 300.0, params)

Lets see what different methods give for frequencies. Note that for each of the options in the method 1 there are 2 types of the eigenvalues/eigenvectors: based on velocity covariance matrix and on the acceleration covariance matrix. For the options in the method 2 there is only 1 type of covariance matrix. So, in all, we have 6 combinations:

In [20]:
ndof = R.num_of_rows
for i in range(ndof):
    print("Mode %i   %5.0f  %5.0f  %5.0f  %5.0f  %5.0f  %5.0f  cm^{-1}" % (i, res00[1].get(i), res00[4].get(i), res01[1].get(i), res01[4].get(i), res20[1].get(i), res21[1].get(i)) )
    

Mode 0    1681    897   1272   1203      0      0  cm^{-1}
Mode 1      13     79      0      0      0      0  cm^{-1}
Mode 2       0      0    420   1131      0      0  cm^{-1}
Mode 3    2675    353   3408    398      0      0  cm^{-1}
Mode 4    1201    257   2076    337      0      0  cm^{-1}
Mode 5    1186    918   1245    893      5      4  cm^{-1}
Mode 6     708    772   1218    952      6      5  cm^{-1}
Mode 7     507    911    713   1076     10     10  cm^{-1}
Mode 8       9    162    527   1244     21     21  cm^{-1}


Now, we can select which normal mode to visualize. To help us with this, we will use the *get_xyz* function in the **normal_modes.py** module. This function combines the geometry (at the first step) and the computed eigenvectors to produce the string with an xyz format data, suitable for visualization with py3Dmol. 

The variables to change are:
* mode - index of the normal mode to visualize
* ampl - the amplitude of magnification factor - just for a better visualization

So, now lets see what the above 6 combinations give us for the modes we have computed

In [21]:
mode = 0
ampl = 500

# Method 1
xyz00v = normal_modes.get_xyz(E,R, M, res00[2], mode)  # velocity covariance
xyz00a = normal_modes.get_xyz(E,R, M, res00[5], mode)  # acceleration covariance
xyz01v = normal_modes.get_xyz(E,R, M, res01[2], mode)  # velocity fluctuation covariance
xyz01a = normal_modes.get_xyz(E,R, M, res01[5], mode)  # acceleration fluctuation covariance

# Method 2
xyz20f = normal_modes.get_xyz(E,R, M, res20[2], mode)  # force covariance
xyz21f = normal_modes.get_xyz(E,R, M, res21[2], mode)  # force fluctuation covariance


view = py3Dmol.view(width=800,height=400, linked=False,viewergrid=(3,2))
view.addModel(xyz00v,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(0,0))
view.addModel(xyz01v,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(0,1))

view.addModel(xyz00a,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(1,0))
view.addModel(xyz01a,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(1,1))

view.addModel(xyz20f,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(2,0))
view.addModel(xyz21f,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(2,1))

view.setBackgroundColor('0xeeeeee')
view.setStyle({'sphere':{}})
view.animate({'loop': 'backAndForth'})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol